In [1]:
import os
import numpy as np
import pandas as pd
from random import random

from agroplot import GoogleMapPlotter

import webbrowser


In [2]:
# Rutas
ROOT_PATH = 'C:/Users/pedro/ds-geo'
CONFIG_PATH = ROOT_PATH + '/config'
MAPS_PATH = ROOT_PATH + '/maps'
DATA_PATH = ROOT_PATH + '/data'

os.chdir(ROOT_PATH)

INITIAL_LAT = 39.720578 
INITIAL_LONG = -2.924675 
INITIAL_ZOOM = 13


# GoogleMaps

In [3]:
class CustomGoogleMapPlotter(GoogleMapPlotter):
    
    # __init__
    def __init__(self, center_lat, center_lng, zoom, apikey='',
                 map_type='satellite'):
        if apikey == '':
            try:
                with open(CONFIG_PATH + '/apikey.txt', 'r') as apifile:
                    apikey = apifile.readline()
            except FileNotFoundError:
                pass
        super().__init__(center_lat, center_lng, zoom, apikey=apikey)

        self.map_type = map_type
        assert(self.map_type in ['roadmap', 'satellite', 'hybrid', 'terrain'])
        

    # write_map
    def write_map(self,  f):
        f.write('\t\tvar centerlatlng = new google.maps.LatLng(%f, %f);\n' %
                (self.center[0], self.center[1]))
        f.write('\t\tvar myOptions = {\n')
        f.write('\t\t\tzoom: %d,\n' % (self.zoom))
        f.write('\t\t\tcenter: centerlatlng,\n')

        # Change this line to allow different map types
        f.write('\t\t\tmapTypeId: \'{}\'\n'.format(self.map_type))
        f.write('\t\t};\n')
        f.write('\t\tvar map = new google.maps.Map(document.getElementById("map_canvas"), myOptions);\n')
        f.write('\n')


    # Añade polígonos y parcelas al mapa
    def AddMapPolygons(self, df, mode=None):

        for index, row in df.iterrows():
            l_lat = row['Latitude']
            l_long = row['Longitude']

            if mode == 'c':
                s_color = row['ColorCultivo']
                if s_color == '':
                    s_color = 'white'
            elif mode == 'p':
                s_color = row['ColorPropietario']
                if s_color == '':
                    s_color = 'white'
            else:
                s_color = 'cornflowerblue'
            
            dict_info = {}
            dict_info.update({'descripcion': row['NombreParcela']})
            dict_info.update({'tipo': row['NombreCultivo']})
            dict_info.update({'tipo_color': row['ColorCultivo']})
            dict_info.update({'variedad': row['VariedadCultivo']})
            dict_info.update({'municipio': row['CatMunicipio']})
            dict_info.update({'poligono': str(row['CatPoligono'])})
            dict_info.update({'parcela': str(row['CatParcela'])})
            dict_info.update({'recinto': str(row['CatRecinto'])})
            dict_info.update({'propietario': row['NombrePropietario']})
            dict_info.update({'propietario_color': row['ColorPropietario']})
           

            self.polygon(l_lat, l_long, info=dict_info, color=s_color, face_alpha=0.5) 

        
        
    # Añade puntos al mapa
    def AddMapPoints(self, df, mode=None):

        for index, row in df.iterrows():
            l_lat = row['Latitude']
            l_long = row['Longitude']
            
            if 'Etiqueta' in row: 
                s_title = row['Etiqueta'] 
            else: 
                s_title = ''
                
            if 'ColorCultivo' in row: 
                s_color = row['ColorCultivo'] 
            else: 
                s_color = 'cornflowerblue'
                
            if mode == 'c':
                if s_color == '': s_color = 'white'
            elif mode == 'p':
                s_color = 'white'
            else:
                s_color = 'cornflowerblue'
            
            dict_info = {}
            dict_info.update({'descripcion': s_title})
            if 'NombreCultivo' in row: dict_info.update({'tipo': row['NombreCultivo']})
            dict_info.update({'tipo_color': s_color})

            self.circle(l_lat, l_long, radius=10, info=dict_info, color=s_color, alpha=0.6)


    # Añade marcadores
    def AddMapMarkers(self, df, mode=None):
        h = ""
        h = h + "<div id='content'>"
        h = h + "<h2 id='firstHeading' class='firstHeading'>Uluru</h2>"
        h = h + "<div id='bodyContent>"
        h = h + "<p><b>Uluru</b></p>"
        h = h + "</div></div>"

        for index, row in df.iterrows():
            l_lat = row['Latitude']
            l_long = row['Longitude']
            s_title = row['Etiqueta'] 
            s_color = row['ColorCultivo'] 
                    
            if mode == 'c':
                if s_color == '': s_color = 'white'
            elif mode == 'p':
                s_color = 'white'
            else:
                s_color = 'cornflowerblue'
            
            dict_info = {}
            dict_info.update({'descripcion': s_title})
            if 'NombreCultivo' in row: dict_info.update({'tipo': row['NombreCultivo']})
            dict_info.update({'tipo_color': s_color})
            
            self.marker(l_lat, l_long, title=s_title, info=dict_info, color=s_color) 

   
    

In [4]:
DATASET_PARCELAS = 'dataset_parcelas.pkl'
DATASET_PUNTOS = 'dataset_puntos.pkl'
DATASET_MARKERS = 'dataset_markers.pkl'
DATASET_MYTRACK_P = 'dataset_mytrack_points.pkl'
DATASET_MYTRACK_M = 'dataset_mytrack_markers.pkl'

# Función que carga el dataset de las parcelas
def LoadDatasetParcelas():
    df = pd.read_pickle(DATA_PATH + '/' + DATASET_PARCELAS)
    return df

# Función que carga el dataset de los puntos de interés
def LoadDatasetPuntos():
    df = pd.read_pickle(DATA_PATH + '/' + DATASET_PUNTOS)
    return df

# Función que carga el dataset de los markers
def LoadDatasetMarkers():
    df = pd.read_pickle(DATA_PATH + '/' + DATASET_MARKERS)
    return df

# Función que carga el dataset de los markers
def LoadDatasetMyTrack():
    df_points = pd.read_pickle(DATA_PATH + '/' + DATASET_MYTRACK_P)
    df_markers = pd.read_pickle(DATA_PATH + '/' + DATASET_MYTRACK_M)
    return (df_points, df_markers)


In [5]:
# Cálculo del centro del mapa y del zoom inicial (para los ficheros de parcelas KML)

def GetMapInitialSizeKML(df):
    
    max_lat = df['Latitude'].apply(max).max()
    min_lat = df['Latitude'].apply(min).min()
    max_long = df['Longitude'].apply(max).max()
    min_long = df['Longitude'].apply(min).min()
    
    center_lat = min_lat + (max_lat - min_lat)/2
    center_long = min_long + (max_long - min_long)/2
    
    width_lat = abs(max_lat - min_lat)
    width_long = abs(max_long - min_long)
    
    zoom = np.ceil(1.3 * (1 / max(width_lat, width_long)))
    zoom = max(zoom, 12)
    zoom = min(zoom, 17)
    
    return (center_lat, center_long, zoom)



In [6]:
# Generación del mapa con los elementos de la entrada
# Se apoya en la clase CustomGoogleMapPlotter, y a su vez en GoogleMapPlotter
# mode: 'C' = cultivo, 'P' = Propietario
# initial: '' = posición y zoom por defecto, '' = cálculo automático

def GenerateMapComp(polygons=None, points=None, markers=None, mode='', initial=''):

    if initial == 'auto':
        initial_lat, initial_long, initial_zoom = GetMapInitialSizeKML(polygons)
        print(initial_lat, initial_long, initial_zoom)
    else:
        initial_lat = INITIAL_LAT
        initial_long = INITIAL_LONG
        initial_zoom = INITIAL_ZOOM
    
    s_mode = mode.lower()
    
    gmap = CustomGoogleMapPlotter(initial_lat, initial_long, initial_zoom, map_type='satellite')
    
    if polygons is not None:
        gmap.AddMapPolygons(polygons, s_mode)    
        
    if points is not None:
        gmap.AddMapPoints(points, s_mode)
    
    if markers is not None:
        gmap.AddMapMarkers(markers)
    
    if s_mode == 'c':
        s_mapname = 'mapa_cultivos.html'
    elif s_mode == 'p':
        s_mapname = 'mapa_propietarios.html'
    else:
        s_mapname = 'mapa.html'
        
    gmap.draw(MAPS_PATH + '/' + s_mapname)    

    # Abrir mapa en el navegador
    url = "file://" + MAPS_PATH + '/' + s_mapname
    webbrowser.open_new_tab(url)


In [7]:
def GenerateMap(layers, mode='', initial=''):

    # Consolido los layers en un único conjunto de dataframes
    polygons = pd.DataFrame(columns=['CatReferencia', 'CatRecinto', 'Latitude', 'Longitude', 'CatMunicipio', 'CatPoligono', 'CatParcela', 'VariedadCultivo', 'NombreParcela', 'NombreCultivo', 'ColorCultivo', 'NombrePropietario', 'ColorPropietario'])
    points = pd.DataFrame(columns=['Latitude', 'Longitude', 'Tipo', 'Etiqueta', 'NombreCultivo', 'ColorCultivo'])
    markers = pd.DataFrame(columns=['Latitude', 'Longitude', 'Tipo', 'Etiqueta', 'NombreCultivo', 'ColorCultivo'])

    for layer in layers:
        if layer.get('parcelas') is not None: polygons = pd.concat([polygons, layer.get('parcelas')], axis=0, ignore_index=True)
        if layer.get('puntos') is not None: points = pd.concat([points, layer.get('puntos')], axis=0, ignore_index=True)
        if layer.get('markers') is not None: markers = pd.concat([markers, layer.get('markers')], axis=0, ignore_index=True)

    polygons.fillna('', inplace=True)
    points.fillna('', inplace=True)
    markers.fillna('', inplace=True)        
        
    #polygons.drop_duplicates(inplace=True, ignore_index=True)
    points.drop_duplicates(inplace=True, ignore_index=True)
    markers.drop_duplicates(inplace=True, ignore_index=True)
        
    # Calculo el marco inicial
    if (initial == 'auto') & (polygons is not None):
        initial_lat, initial_long, initial_zoom = GetMapInitialSizeKML(polygons)
        print(initial_lat, initial_long, initial_zoom)
    else:
        initial_lat = INITIAL_LAT
        initial_long = INITIAL_LONG
        initial_zoom = INITIAL_ZOOM
    
    s_mode = mode.lower()
    
    # Genero el mapa
    gmap = CustomGoogleMapPlotter(initial_lat, initial_long, initial_zoom, map_type='satellite')
    
    if polygons is not None:
        gmap.AddMapPolygons(polygons, s_mode)    
        
    if points is not None:
        gmap.AddMapPoints(points, s_mode)
    
    if markers is not None:
        gmap.AddMapMarkers(markers, s_mode)
    
    if s_mode == 'c':
        s_mapname = 'mapa_cultivos.html'
    elif s_mode == 'p':
        s_mapname = 'mapa_propietarios.html'
    else:
        s_mapname = 'mapa.html'
        
    gmap.draw(MAPS_PATH + '/' + s_mapname)    

    # Abrir mapa en el navegador
    url = "file://" + MAPS_PATH + '/' + s_mapname
    webbrowser.open_new_tab(url)


### Ejecución

In [8]:
# Carga de las parcelas
df_parcelas = LoadDatasetParcelas()

In [9]:
# Carga puntos destacados
df_puntos = LoadDatasetPuntos()

In [10]:
# Carga de los marcadores
df_markers = LoadDatasetMarkers()

In [11]:
# Carga de los datos de la app MyTrack
df_mytrack_p, df_mytrack_m = LoadDatasetMyTrack()

In [12]:
layer_general = {}
layer_general.update({'parcelas': df_parcelas})
layer_general.update({'puntos': df_puntos})
layer_general.update({'markers': df_markers})

layer_mytrack = {}
layer_mytrack.update({'puntos': df_mytrack_p})
layer_mytrack.update({'markers': df_mytrack_m})

layers = [layer_general, layer_mytrack]

GenerateMap(layers, mode='c')


In [13]:
#p = df_parcelas[(df_parcelas['Ref']=='16259A037001760000ZX') & (df_parcelas['Sub'] == 'f')]
#GenerateMapComp(polygons = p)

#df_parcelas[['Ref', 'Sub']][df_parcelas['VariedadCultivo'] == '']

In [14]:
df_markers

,Latitude,Longitude,Tipo,Etiqueta,NombreCultivo,ColorCultivo
0,39.741266,-2.933473,INM,Marker inicio,Instalaciones e inmuebles,red
1,39.725870,-2.939457,OTR,Marker aleatorio 1,Otros,white
2,39.688158,-2.938821,CAM,Cruce importante,Caminos y carriles,white
3,39.734373,-2.931537,CZA,Aparcamiento del coto,Caza,darkolivegreen
